In [1]:
from sentence_transformers import SentenceTransformer, util
import pm4py
from pm4py.objects.bpmn.obj import BPMN
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [3]:
bpmn = pm4py.read_bpmn('V_G01_text.bpmn')
temp=[]
for flow in bpmn.get_flows():
    temp.append([flow.source,flow.target,flow.get_name()])

for rel in temp:
    if(isinstance(rel[1],BPMN.Gateway) and len(rel[1].get_in_arcs())>1):
        rel[1]=rel[1].get_out_arcs()[0].target
        while(isinstance(rel[1],BPMN.Gateway)):
            rel[1]=rel[1].get_out_arcs()[0].target

temp1=[]
for rel in temp:
    if(rel[0]==rel[1]):
        continue
    elif(isinstance(rel[0],BPMN.Gateway) and len(rel[0].get_in_arcs())>1):
        continue
    else:
        temp1.append(rel)
df_BPMN = pd.DataFrame(columns=['Source', 'Target', 'Relation'])
for rel in temp1:
    if(isinstance(rel[1],BPMN.EndEvent) or isinstance(rel[0],BPMN.StartEvent)):
        continue
    if(isinstance(rel[0],BPMN.ExclusiveGateway)):
        # temp2.append([rel[0].get_in_arcs()[0].source, rel[1]])
        df_BPMN = df_BPMN.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'XOR_gateway'}, ignore_index=True)
        df_BPMN = df_BPMN.append({'Source': rel[2], 'Target': rel[1].name, 'Relation': 'XOR_condition'}, ignore_index=True)
        # print([rel[0].get_in_arcs()[0].source, rel[1]])
    elif(isinstance(rel[1],BPMN.ExclusiveGateway)):
        continue
    elif(isinstance(rel[0],BPMN.ParallelGateway)):
        df_BPMN = df_BPMN.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'parallel_gateway'}, ignore_index=True)
    elif(isinstance(rel[1],BPMN.ParallelGateway)):
        continue
    else:
        df_BPMN = df_BPMN.append({'Source': rel[0].name, 'Target': rel[1].name, 'Relation': 'Flow'}, ignore_index=True)

In [4]:
# log = pm4py.read_xes("R_j04_event.xes")
# bpmn_event = pm4py.discover_bpmn_inductive(log, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
bpmn_event = pm4py.read_bpmn('V_G01_event.bpmn')
temp=[]
for flow in bpmn_event.get_flows():
    temp.append([flow.source,flow.target,flow.get_name()])
for rel in temp:
    if(isinstance(rel[1],BPMN.Gateway) and len(rel[1].get_in_arcs())>1):
        rel[1]=rel[1].get_out_arcs()[0].target
        while(isinstance(rel[1],BPMN.Gateway)):
            rel[1]=rel[1].get_out_arcs()[0].target

temp1=[]
for rel in temp:
    if(rel[0]==rel[1]):
        continue
    elif(isinstance(rel[0],BPMN.Gateway) and len(rel[0].get_in_arcs())>1):
        continue
    else:
        temp1.append(rel)
df_BPMN_event = pd.DataFrame(columns=['Source', 'Target', 'Relation'])
for rel in temp1:
    if(isinstance(rel[1],BPMN.EndEvent) or isinstance(rel[0],BPMN.StartEvent)):
        continue
    if(isinstance(rel[0],BPMN.ExclusiveGateway)):
        # temp2.append([rel[0].get_in_arcs()[0].source, rel[1]])
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'XOR_gateway'}, ignore_index=True)
        df_BPMN_event = df_BPMN_event.append({'Source': rel[2], 'Target': rel[1].name, 'Relation': 'XOR_condition'}, ignore_index=True)
        # print([rel[0].get_in_arcs()[0].source, rel[1]])
    elif(isinstance(rel[1],BPMN.ExclusiveGateway)):
        continue
    elif(isinstance(rel[0],BPMN.ParallelGateway)):
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'parallel_gateway'}, ignore_index=True)
    elif(isinstance(rel[1],BPMN.ParallelGateway)):
        continue
    else:
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].name, 'Target': rel[1].name, 'Relation': 'Flow'}, ignore_index=True)

In [5]:
list_event=[]
for row in df_BPMN_event.itertuples(index=False):
    if(row.Relation!='XOR_condition'):
        list_event.append((row.Source,row.Target,row.Relation))
list_BPMN=[]
for row in df_BPMN.itertuples(index=False):
    if(row.Relation!='XOR_condition'):
        list_BPMN.append((row.Source,row.Target,row.Relation))
dif_gold=set(list(set(list_BPMN).difference(set(list_event)))+list(set(list_event).difference(set(list_BPMN))))

In [6]:
log = pm4py.read_xes("V_G01_event.xes")
bpmn_event = pm4py.discover_bpmn_inductive(log, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
# bpmn_event = pm4py.read_bpmn('R_j04_event.bpmn')
temp=[]
for flow in bpmn_event.get_flows():
    temp.append([flow.source,flow.target,flow.get_name()])
for rel in temp:
    if(isinstance(rel[1],BPMN.Gateway) and len(rel[1].get_in_arcs())>1):
        rel[1]=rel[1].get_out_arcs()[0].target
        while(isinstance(rel[1],BPMN.Gateway)):
            rel[1]=rel[1].get_out_arcs()[0].target

temp1=[]
for rel in temp:
    if(rel[0]==rel[1]):
        continue
    elif(isinstance(rel[0],BPMN.Gateway) and len(rel[0].get_in_arcs())>1):
        continue
    else:
        temp1.append(rel)
df_BPMN_event = pd.DataFrame(columns=['Source', 'Target', 'Relation'])
for rel in temp1:
    if(isinstance(rel[1],BPMN.EndEvent) or isinstance(rel[0],BPMN.StartEvent)):
        continue
    if(isinstance(rel[0],BPMN.ExclusiveGateway)):
        # temp2.append([rel[0].get_in_arcs()[0].source, rel[1]])
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'XOR_gateway'}, ignore_index=True)
        df_BPMN_event = df_BPMN_event.append({'Source': rel[2], 'Target': rel[1].name, 'Relation': 'XOR_condition'}, ignore_index=True)
        # print([rel[0].get_in_arcs()[0].source, rel[1]])
    elif(isinstance(rel[1],BPMN.ExclusiveGateway)):
        continue
    elif(isinstance(rel[0],BPMN.ParallelGateway)):
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'parallel_gateway'}, ignore_index=True)
    elif(isinstance(rel[1],BPMN.ParallelGateway)):
        continue
    else:
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].name, 'Target': rel[1].name, 'Relation': 'Flow'}, ignore_index=True)

c:\Users\20235451\.conda\envs\trochtest\lib\site-packages\pm4py\util\dt_parsing\parser.py:78: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  "ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11")


parsing log, completed traces ::   0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
model = SentenceTransformer("Maite89/Roberta_finetuning_semantic_similarity_stsb_multi_mt")
Activities_text=[]
relations=[('robot receives an order', 'asks whether the customer wants a menu or just the burger', 'Flow'),
('asks whether the customer wants a menu or just the burger', 'starts preparing the foods', 'XOR_gateway'),
('customer wants a menu', 'starts preparing the foods', 'XOR_condition'),
('asks whether the customer wants a menu or just the burger', 'prepares the burger', 'XOR_gateway'),
('customer wants just the burger', 'prepares the burger', 'XOR_condition'),
('starts preparing the foods', 'preparing the drink', 'Flow'),
('preparing the drink', 'asks if you want fries or wedges', 'Flow'),
('asks if you want fries or wedges', 'prepares fries', 'XOR_gateway'),
('wants fries', 'prepares fries', 'XOR_condition'),
('asks if you want fries or wedges', 'prepares wedges', 'XOR_gateway'),
('wants wedges', 'prepares wedges', 'XOR_condition'),
('prepares fries', 'prepares the burger', 'Flow'),
('prepares wedges', 'prepares the burger', 'Flow'),
('prepares the burger', 'gives enthusiastic status updates every 30 seconds', 'Flow'),
('gives enthusiastic status updates every 30 seconds', 'order is delivered via a conveyor belt', 'Flow')]
df_text = pd.DataFrame(columns=['Source', 'Target', 'Relation'])
for relation in relations:
    df_text = df_text.append({'Source': relation[0], 'Target': relation[1], 'Relation': relation[2]}, ignore_index=True)
    Activities_text.append(relation[1])
    if relation[2]!='XOR_condition':
        Activities_text.append(relation[0])
queries=list(set(Activities_text))

In [8]:
Activities_BPMN=[]
for node in bpmn_event.get_nodes():
    if(isinstance(node,BPMN.EndEvent) or isinstance(node,BPMN.StartEvent)):
        continue
    if(isinstance(node,BPMN.Activity) or isinstance(node,BPMN.Event)):
        Activities_BPMN.append(node.name)

# entity match
df = pd.DataFrame(columns=['text', 'BPMN', 'score'])
temp_list=[]
for query in queries:
    query_embedding = torch.FloatTensor(model.encode(query))
    scores=[]
    for activity in Activities_BPMN:
        activity_embedding = torch.FloatTensor(model.encode(activity))
        cos_sim = F.cosine_similarity(query_embedding, activity_embedding, dim=0)
        scores.append(cos_sim.item())
    # print(scores)
    max_score_pos=np.argmax(scores)
    max_score=np.max(scores)
    if(max_score<0.5):
        df = df.append({'text': query, 'BPMN': "No match found", 'score': max_score}, ignore_index=True)
    else:
        if(max_score_pos in temp_list):
            score_tep=df.loc[df['BPMN']==Activities_BPMN[max_score_pos]]['score'].tolist()[0]
            if(max_score>score_tep):
                df.loc[df['BPMN']==Activities_BPMN[max_score_pos],'BPMN']="No match found"
                df = df.append({'text': query, 'BPMN': Activities_BPMN[max_score_pos], 'score': max_score}, ignore_index=True)
            else:
                df = df.append({'text': query, 'BPMN': "No match found", 'score': max_score}, ignore_index=True)
        else:
            temp_list.append(max_score_pos)
            df = df.append({'text': query, 'BPMN': Activities_BPMN[max_score_pos], 'score': max_score}, ignore_index=True)
for i in range(len(Activities_BPMN)):
    if i not in temp_list:
        df = df.append({'text': "No match found", 'BPMN': Activities_BPMN[i], 'score': 0}, ignore_index=True)

In [9]:
# replace
df_text_new = pd.DataFrame(columns=['Source', 'Target', 'Relation'])
for row in df_text.itertuples(index=False):
    source=row.Source
    target=row.Target
    relation=row.Relation
    if(source!="No match found"):
        if(len(df.loc[df['text']==source]['BPMN'].tolist())>0):
            if(df.loc[df['text']==source]['BPMN'].tolist()[0]!="No match found"):
                source=df.loc[df['text']==source]['BPMN'].tolist()[0]
    if(target!="No match found"):
        if(len(df.loc[df['text']==target]['BPMN'].tolist())>0):
            if(df.loc[df['text']==target]['BPMN'].tolist()[0]!="No match found"):
                target=df.loc[df['text']==target]['BPMN'].tolist()[0]
    df_text_new = df_text_new.append({'Source': source, 'Target': target, 'Relation': relation}, ignore_index=True)

list_text=[]
for row in df_text_new.itertuples(index=False):
    if(row.Relation!='XOR_condition'):
        list_text.append((row.Source,row.Target,row.Relation))
list_event=[]
for row in df_BPMN_event.itertuples(index=False):
    if(row.Relation!='XOR_condition'):
        list_event.append((row.Source,row.Target,row.Relation))
dif=set(list(set(list_text).difference(set(list_event)))+list(set(list_event).difference(set(list_text))))
precision=len(dif.intersection(dif_gold))/(len(dif))
recall=len(dif.intersection(dif_gold))/len(dif_gold)
print('precision:',precision)
print('recall:',recall)

precision: 0.5882352941176471
recall: 0.7142857142857143


In [10]:
# detected deviation relationships
dif

{('Ask about side dish', 'prepare fries', 'Flow'),
 ('Check whether the customer wants a menu or a burger',
  'deliver order using convey or belt',
  'XOR_gateway'),
 ('Check whether the customer wants a menu or a burger',
  'prepare burger',
  'XOR_gateway'),
 ('Start preparing foods', 'Ask about side dish', 'parallel_gateway'),
 ('Start preparing foods', 'prepare drink', 'Flow'),
 ('Start preparing foods', 'prepare drink', 'parallel_gateway'),
 ('asks if you want fries or wedges', 'prepare fries', 'XOR_gateway'),
 ('asks if you want fries or wedges', 'prepares wedges', 'XOR_gateway'),
 ('deliver order using convey or belt', 'give status updates', 'Flow'),
 ('give status updates', 'deliver order using convey or belt', 'Flow'),
 ('give status updates', 'prepare burger', 'Flow'),
 ('prepare burger', 'give status updates', 'Flow'),
 ('prepare drink', 'asks if you want fries or wedges', 'Flow'),
 ('prepare drink', 'deliver order using convey or belt', 'Flow'),
 ('prepare fries', 'deliver 